In [1]:
from ece470_lib import *
import numpy as np
from numpy.linalg import inv, norm

In [2]:
def collision_check(pts, radii, pts2, radii2, dprint=False):
    Np = 0
    Np = pts.shape[1]
    pts = np.hsplit(pts, Np)
    radii = radii.flatten()
    for p1, r1 in zip(pts, radii):
        for p2, r2 in zip(pts, radii):
            if p1 is not p2:
                if norm(p1 - p2) < r1 + r2:
                    if dprint:
                        print("self col")
                    return True
                
    pts2 = np.hsplit(pts2, pts2.shape[1])
    radii2 = radii2.flatten()
    for p1, r1 in zip(pts, radii):
        for p2, r2, in zip(pts2, radii2):
            if norm(p1 - p2) <= r1 + r2:
                if dprint:
                    print(p1)
                    print("")
                    print(p2)
                    print("obstacle col")
                return True
    return False

def random_theta(min_val=0, max_val=1, shape=(1,)):
    return (max_val - min_val)*np.random.random_sample(shape)+min_val

def collision_check_line(t_start, t_end, p_robot, r_robot, S, p_obstacle, r_obstacle, num=100):
    for theta in matrix_linspace(t_start, t_end, num, to_end=True):
        tpts = multi_transform(p_robot, S, theta)
        if collision_check(tpts, r_robot, p_obstacle, r_obstacle):
            return True
    return False

In [3]:
#1

In [3]:
import numpy as np

S = np.array([[0.00, 0.00], [0.00, 1.00], [0.00, 0.00], [0.00, -6.00], [0.00, 0.00], [1.00, 0.00]])
M = np.array([[0.00, 0.00, 1.00, 2.00], [0.00, 1.00, 0.00, 0.00], [-1.00, 0.00, 0.00, 6.00], [0.00, 0.00, 0.00, 1.00]])
p_robot = np.array([[0.00, 0.00, 0.00, 2.00], [0.00, 0.00, 0.00, 0.00], [0.00, 2.00, 6.00, 6.00]])
r_robot = np.array([[0.90, 0.90, 0.90, 0.90]])
p_obstacle = np.array([[1.06, 4.75, 1.03, -3.83, 3.83, 2.91, -2.07, 4.66, 2.40, 1.19, 2.36,  4.76, -3.78, -4.16, -3.19, 3.84, -4.38, -4.77, 4.36, -4.31, -3.70,  -1.21, 4.12, 1.86, -4.53, 4.08, 0.06, -3.81, -3.35, 2.55, -0.87, 0.55], [4.50, 2.27, -3.10, 2.04, -3.94, -0.71, -3.94, 2.92, -3.59, -3.78, -3.77,  -1.49, -1.86, -4.57, -3.94, 3.89, 1.04, -1.09, 1.01, 1.10, -2.88, -0.88,  -2.13, 3.90, -0.61, 1.12, 3.86, 4.91, -0.65, 4.82, 3.89, 2.51], [-0.99, 3.24, 4.39, -3.68, -3.33, -2.42, 3.95, -0.16, 2.40, 3.32, -3.29,  2.30, 2.17, -3.78, 0.64, 1.55, 4.81, -1.50, -3.11, 0.56, 0.30, -4.93,  4.52, -0.88, -3.52, -2.97, 0.85, 3.87, -4.24, -2.41, 1.20, 4.11]])
r_obstacle = np.array([[1.96, 2.56, 0.57, 0.88, 4.47, 2.61, 3.40, 1.16, 1.91, 0.99, 3.83, 3.09,  2.91, 3.54, 2.31, 4.14, 1.15, 0.54, 1.85, 1.80, 2.23, 0.61, 0.89, 3.08,  3.25, 2.35, 1.95, 1.66, 3.34, 2.71, 0.63, 1.80]])
theta_start = np.array([[-0.64], [0.23]])
theta_goal = np.array([[1.65], [3.13]])


In [4]:
ts = Tree(theta_start)
te = Tree(theta_goal)
N = S.shape[1]
max_iter = 100
path_found = False
while max_iter > 0 and not path_found:
    max_iter -= 1
    r_theta = random_theta(min_val=-np.pi, max_val=np.pi, shape=theta_start.shape)
    min_theta_start = None
    b1 = False
    b2 = False
    min_dist = 100000
    for t in ts:
        if norm(t-r_theta) < min_dist:
            min_theta_start = t
            min_dist = norm(t-r_theta)
    if min_theta_start is not None and not collision_check_line(min_theta_start, r_theta, p_robot, r_robot, S, p_obstacle, r_obstacle):
        ts.insert(r_theta, min_theta_start)
        b1 = True
    min_dist = 100000
    for t in te:
        if norm(t-r_theta) < min_dist:
            min_theta_end = t
            min_dist = norm(t-r_theta)
    if min_theta_end is not None and not collision_check_line(min_theta_end, r_theta, p_robot, r_robot, S, p_obstacle, r_obstacle):
        te.insert(r_theta, min_theta_end)
        b2 = True
    path_found = b1 and b2
if max_iter is 0:
    print("Failed")

ret = [r_theta]
app = ts.parent(r_theta)
while app is not None:
    ret.insert(0, app)
    app = ts.parent(app)
ret.insert(0, theta_start)
app = te.parent(r_theta)
while app is not None:
    ret.append(app)
    app = te.parent(app)
ret.append(theta_goal)
ret

[array([[-0.64],
        [ 0.23]]), array([[-0.64],
        [ 0.23]]), array([[  6.04649591e-01],
        [  2.23023994e-05]]), array([[ 1.65],
        [ 3.13]]), array([[ 1.65],
        [ 3.13]])]

In [5]:
np.hstack(ret)

array([[ -6.40000000e-01,  -6.40000000e-01,   6.04649591e-01,
          1.65000000e+00,   1.65000000e+00],
       [  2.30000000e-01,   2.30000000e-01,   2.23023994e-05,
          3.13000000e+00,   3.13000000e+00]])

In [ ]:
#2

In [6]:
import numpy as np

S = np.array([[0.00, 1.00, 0.00, 0.00, 0.00, -1.00], [0.00, 0.00, 0.00, 1.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00, 0.00, 0.00], [1.00, 0.00, 0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 1.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 4.00, -1.00, 2.00]])
M = np.array([[0.00, 0.00, -1.00, 0.00], [1.00, 0.00, 0.00, 2.00], [0.00, -1.00, 0.00, 0.00], [0.00, 0.00, 0.00, 1.00]])
p_robot = np.array([[0.00, 2.00, 4.00, 4.00, 4.00, 2.00, 2.00, 0.00], [0.00, 0.00, 0.00, 2.00, 4.00, 4.00, 2.00, 2.00], [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]])
r_robot = np.array([[0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90]])
p_obstacle = np.array([[-1.21, 1.53, 1.72, 1.47, -0.14, -2.10, -4.37, -3.41, -1.78, -0.78, 0.05,  0.80, -4.22, 2.17, 4.56], [-4.39, -3.39, -4.58, -0.23, -4.75, 3.63, 1.79, 3.20, -1.91, -4.98,  -2.88, 1.87, -4.44, -3.10, 1.29], [-0.22, -3.73, 4.00, 3.40, -4.71, 2.36, 3.66, -4.46, -1.56, 2.54, 4.61,  -2.75, -1.07, -0.32, 4.80]])
r_obstacle = np.array([[3.24, 1.76, 2.74, 1.56, 0.54, 0.53, 3.86, 3.02, 1.10, 1.00, 0.64, 0.73,  0.93, 1.61, 3.49]])
theta_start = np.array([[2.74], [-1.62], [2.09], [-0.56], [-1.75], [-0.67]])
theta_goal = np.array([[0.74], [0.12], [2.08], [-0.23], [0.99], [-1.89]])


In [7]:
ts = Tree(theta_start)
te = Tree(theta_goal)
N = S.shape[1]
max_iter = 100
path_found = False
while max_iter > 0 and not path_found:
    max_iter -= 1
    r_theta = random_theta(min_val=-np.pi, max_val=np.pi, shape=theta_start.shape)
    min_theta_start = None
    b1 = False
    b2 = False
    min_dist = 100000
    for t in ts:
        if norm(t-r_theta) < min_dist:
            min_theta_start = t
            min_dist = norm(t-r_theta)
    if min_theta_start is not None and not collision_check_line(min_theta_start, r_theta, p_robot, r_robot, S, p_obstacle, r_obstacle):
        ts.insert(r_theta, min_theta_start)
        b1 = True
    min_dist = 100000
    for t in te:
        if norm(t-r_theta) < min_dist:
            min_theta_end = t
            min_dist = norm(t-r_theta)
    if min_theta_end is not None and not collision_check_line(min_theta_end, r_theta, p_robot, r_robot, S, p_obstacle, r_obstacle):
        te.insert(r_theta, min_theta_end)
        b2 = True
    path_found = b1 and b2
if max_iter is 0:
    print("Failed")

ret = [r_theta]
app = ts.parent(r_theta)
while app is not None:
    ret.insert(0, app)
    app = ts.parent(app)
ret.insert(0, theta_start)
app = te.parent(r_theta)
while app is not None:
    ret.append(app)
    app = te.parent(app)
ret.append(theta_goal)
np.hstack(ret)

array([[ 2.74      ,  2.74      ,  2.5697111 ,  0.74      ,  0.74      ],
       [-1.62      , -1.62      , -0.61583896,  0.12      ,  0.12      ],
       [ 2.09      ,  2.09      ,  3.09454901,  2.08      ,  2.08      ],
       [-0.56      , -0.56      , -1.80417892, -0.23      , -0.23      ],
       [-1.75      , -1.75      , -2.9939805 ,  0.99      ,  0.99      ],
       [-0.67      , -0.67      , -2.35084772, -1.89      , -1.89      ]])

In [ ]:
ts.getElements()